In [31]:
import keras
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.utils import to_categorical
from keras.initializers import RandomUniform

%matplotlib inline

In [7]:
boneData = pd.read_csv('boneage-training-dataset.csv')
paths = []
print(len(boneData['id']))
boneData = boneData[(boneData.boneage <= 4.*12) | (boneData.boneage >= 15.5*12)]
for i in boneData['id']:
    paths.append('boneage-training-dataset/' + str(i) + '.png')
print(len(paths))

12611
1240


In [8]:
X = []
for path in paths:
    img = image.load_img(path, target_size=(224,224))
    img = image.img_to_array(img)
    X.append(img)

In [9]:
Y = np.array(boneData['boneage'])
#print(Y.max())
#print(np.median(Y))
#print(Y.min())

print(Y[0:10])

pos = 0
for i in range(0,len(Y)):
    if(Y[i]/12)< 11: 
        Y[i] = 0
    else: 
        Y[i] = 1
        pos += 1
        
print(Y[0:10])
print(float(pos/len(Y)))
split = float(pos/len(Y))
Y = to_categorical(Y)

[ 12  36 188   4  36  24  30  33  24  32]
[0 0 1 0 0 0 0 0 0 0]
0.5419354838709678


In [13]:
yTrain = Y[:1000]
yVal = Y[1000:]

In [11]:
xTrain = np.array(X)

In [12]:
xTrain = xTrain/255.
xTrain = xTrain.astype(np.float32)

In [14]:
xVal = xTrain[1000:]
xTrain = xTrain[:1000]

In [15]:
x = xTrain
x = preprocess_input(x)

In [19]:
model = VGG16(weights='imagenet', include_top=True)

In [20]:
model2 = VGG16(weights=None, include_top = True, classes = 2)

In [32]:
kinit = RandomUniform()

In [43]:
kmodel = Sequential([
    Flatten(input_shape = (224, 224, 3)),
    Dense(1000, activation = 'relu', kernel_initializer = kinit),
    Dense(500, activation = 'relu', kernel_initializer = kinit),
    Dense(750, activation = 'relu', kernel_initializer = kinit),
    Dense(200, activation = 'relu', kernel_initializer = kinit),
    Dense(100, activation = 'relu', kernel_initializer = kinit),
    Dense(50, activation = 'relu', kernel_initializer = kinit),
    Dense(25, activation = 'relu', kernel_initializer = kinit),
    Dense(10, activation = 'relu', kernel_initializer = kinit),
    Dense(2, activation = 'softmax', kernel_initializer = kinit)
])

In [44]:
sgd = keras.optimizers.SGD(lr = .01)
kmodel.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics=['accuracy'], )

In [ ]:
h = kmodel.fit(xTrain, yTrain, epochs = 10, validation_data=[xVal, yVal], class_weight = model.weights, verbose = 1)

Train on 1000 samples, validate on 240 samples
Epoch 1/10
1000/1000 [==============================] - 46s 46ms/step - loss: 0.6926 - acc: 0.5350 - val_loss: 0.6917 - val_acc: 0.5708
Epoch 2/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.6918 - acc: 0.5353

In [98]:
print(h.history['acc'])
print(split)

[0.5687, 0.5687, 0.5687]
0.5640314011577194


In [ ]:
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.compile(optimizer = sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [99]:
vgg162 = model2.fit(x=xTrain, y=yTrain, validation_data=[xVal,yVal], epochs=3, class_weight = model.weights, batch_size = 128)

Train on 10000 samples, validate on 2611 samples
Epoch 1/3
10000/10000 [==============================] - 22079s 2s/step - loss: 9.1664 - acc: 0.4313 - val_loss: 8.8029 - val_acc: 0.4538
Epoch 2/3
10000/10000 [==============================] - 19209s 2s/step - loss: 9.1664 - acc: 0.4313 - val_loss: 8.8029 - val_acc: 0.4538
Epoch 3/3
10000/10000 [==============================] - 36962s 4s/step - loss: 9.1664 - acc: 0.4313 - val_loss: 8.8029 - val_acc: 0.4538


In [ ]:
print(vggh162.history['acc'])
print(vggh162.history['val_acc'])